# LangChain

- What is LangChain?
- Type of prompts:
	- Simple Prompt
	- Message Prompt
- Initialize LLM
- Execute a prompt with LLM
- What's included in LangChain
- LangChain Expression Language (LCEL)
- AI Agent
- Structured Output


---
## Simple Prompt in LangChain

In [1]:
## Prompt simple way
import pprint

print("------------Normal Way-----------------------")
prompt = "Answer the user query in one line.\n What is Gen AI?"


pprint.pprint(prompt)


## LangChain way

print("------------LangChain way-----------------------")
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="Answer the user query in one line.\n{query}\n",
    input_variables=["query"],
)

text_prompt = prompt.invoke({"query": "What is Gen AI?"})

pprint.pprint(text_prompt)

------------Normal Way-----------------------
'Answer the user query in one line.\n What is Gen AI?'
------------LangChain way-----------------------
StringPromptValue(text='Answer the user query in one line.\nWhat is Gen AI?\n')


---
## Message Prompt in LangChain
This is new way of prompting in LLM. This follows simple chat like structure.
```
Me: How are you?
LLM: I am fine.
```

In [2]:
## Chat Message simple way
import pprint

print("------------Normal Way-----------------------")
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

pprint.pprint(messages)


## LangChain way

print("------------LangChain way-----------------------")

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that translates English to {language}. Translate the user sentence."),
    MessagesPlaceholder("msgs")
])

prompt_messages = prompt_template.invoke({
    "msgs": [HumanMessage(content="I love programming.")],
    "language": "Hindi"
})

for message in prompt_messages.messages:
    message.pretty_print()

------------Normal Way-----------------------
[('system',
  'You are a helpful assistant that translates English to French. Translate '
  'the user sentence.'),
 ('human', 'I love programming.')]
------------LangChain way-----------------------
================================ System Message ================================

You are a helpful assistant that translates English to Hindi. Translate the user sentence.
================================ Human Message =================================

I love programming.


---
## Initialize LLM
- LangChain provides different packages for integrating with different LLM's. 
- List can be seen in [Chat Model](https://python.langchain.com/docs/integrations/chat/)
- Not all Chat Model support all the feature, so make sure you check if it's supported before using it.

In [3]:
## initialize DeepSeek LLM

from langchain_deepseek import ChatDeepSeek

deepseek_llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [4]:
## initialize OpenAI LLM

from langchain_openai import ChatOpenAI

openai_llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

---
## Execute a prompt with LLM

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_deepseek import ChatDeepSeek


# Initialize a simple prompt
prompt = PromptTemplate(
    template="Answer the user query in one line.\n{query}\n",
    input_variables=["query"],
)

text_prompt = prompt.invoke({"query": "What is Gen AI?"})

## initialize DeepSeek LLM
deepseek_llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

## Execute a prompt with LLM
response = deepseek_llm.invoke(text_prompt)
response.pretty_print()

================================== Ai Message ==================================

Gen AI is artificial intelligence that generates new content like text, images, or code based on input data.


In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

# Initialize a chat prompt template
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that translates English to {language}. Translate the user sentence."),
    MessagesPlaceholder("msgs")
])

prompt_messages = prompt_template.invoke({
    "msgs": [HumanMessage(content="I love programming.")],
    "language": "French"
})

## initialize DeepSeek LLM
deepseek_llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

## Execute a prompt with LLM
response = deepseek_llm.invoke(prompt_messages)
response.pretty_print()

================================== Ai Message ==================================

J'adore programmer.


---
## LangChain Expression Language (LCEL)
LCEL allows to write more readable code.

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain_deepseek import ChatDeepSeek

# Initialize a chat prompt template
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that translates English to {language}. Translate the user sentence."),
    MessagesPlaceholder("msgs")
])

# prompt_messages = prompt_template.invoke({
#     "msgs": [HumanMessage(content="I love programming.")],
#     "language": "French"
# })

## initialize DeepSeek LLM
deepseek_llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

chained_LLM  = prompt_template | deepseek_llm

## Execute a prompt with LLM
response = chained_LLM.invoke({
    "msgs": [HumanMessage(content="I love programming.")],
    "language": "French"
})
response.pretty_print()

================================== Ai Message ==================================

J'adore la programmation.


---
## AI Agent
- AI agent can not only use it's knowledge to answer the question, but it can also act on some task. 
- LLM can be equipped with many tools can perform tasks
- Check support for tool calling before using with LLM
- There are some open source [Tools](https://python.langchain.com/docs/integrations/tools/) available

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool

## Setting up Tools
@tool
def check_email(name:str = None) -> str:
    """Use this tool to get users email address."""
    return "janedoe@example.com"

## Initialize DeepSeek LLM
deepseek_llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

## Bind tools to the DeepSeek LLM
deepseek_with_tools = deepseek_llm.bind_tools([check_email])

## Create a prompt template for the chat
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("msgs")
])
prompt_messages = prompt_template.invoke({
    "msgs": [HumanMessage(content="What is Vimal Menon email address.")]
})

## Invoke the DeepSeek LLM with tools
response = deepseek_with_tools.invoke(prompt_messages)
prompt_messages.messages.append(response)
for tool in response.tool_calls:
    if tool["name"] == "check_email":
        tool_msg = check_email.invoke(tool["args"])
        prompt_messages.messages.append(ToolMessage(tool_msg, tool_call_id=tool["id"]))
response = deepseek_with_tools.invoke(prompt_messages)
prompt_messages.messages.append(response)

for message in prompt_messages.messages:
    message.pretty_print()


================================ System Message ================================

You are a helpful assistant.
================================ Human Message =================================

What is Vimal Menon email address.
================================== Ai Message ==================================
Tool Calls:
  check_email (call_0_662834ab-cf90-4e64-8e29-d164c29a7c13)
 Call ID: call_0_662834ab-cf90-4e64-8e29-d164c29a7c13
  Args:
    name: Vimal Menon
================================= Tool Message =================================

janedoe@example.com
================================== Ai Message ==================================

The email address for Vimal Menon is janedoe@example.com.


---
## Structured Output

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_deepseek import ChatDeepSeek

from pydantic import BaseModel, Field
from pprint import pprint



## Initialize DeepSeek LLM
deepseek_llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Pydantic
class Country(BaseModel):
    """List countries detail."""
    name: str = Field(description="The name of the country")
    capital: str = Field(description="The capital city of the country")
    population: int = Field(description="The population of the country")
    states: list[str] = Field(description="List the states in the country")


class CountryList(BaseModel):
    """List of countries."""		
    countries: list[Country] = Field(description="List of countries with their capitals and populations")


deepseek_llm_with_output = deepseek_llm.with_structured_output(CountryList)

## Create a prompt template for the chat
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("msgs")
])
prompt_messages = prompt_template.invoke({
    "msgs": [HumanMessage(content="List out top 5 countries as per their population.")]
})

deepseek_response = deepseek_llm_with_output.invoke(prompt_messages)

pprint(deepseek_response)

CountryList(countries=[Country(name='China', capital='Beijing', population=1439323776, states=['Anhui', 'Fujian', 'Gansu', 'Guangdong', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan', 'Hubei', 'Hunan', 'Jiangsu', 'Jiangxi', 'Jilin', 'Liaoning', 'Qinghai', 'Shaanxi', 'Shandong', 'Shanxi', 'Sichuan', 'Yunnan', 'Zhejiang']), Country(name='India', capital='New Delhi', population=1380004385, states=['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']), Country(name='United States', capital='Washington, D.C.', population=331002651, states=['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', '